# Stereo Vision Pipeline

In this example, a complete stereo vision pipeline after calibration is implemented.

![](pipeline.png)

From the dataset SceneNN [1], two images had been captured with two cameras that are not well aligned. Hence, a rectification has to preprocess the data. For the sake of simplicity, assume that the camera orientataions of $\mathbf{C}_\text{left}$ and $\mathbf{C}_\text{right}$ only differ in a rotation around their z-axes.
After rectification, the *Semi Global Block Matching* algorithm of [2] is used in order to find the point correspondences. The result, the disparity map, has to be transfered to the depth map ($z$ map).
Finally, a point cloud will be generated using the depth information.

In [ ]:
# some libraries we will need later on...

import cv2
import numpy as np
import os

Let's define some helper methods. `rot_z` builds up a 2D rotation homography for rotating an image around the origin. The method `calc_r_rect` builds up the rectification homography. For more details on `calc_r_rect`, please consider the slides of Chapter 7.

In [ ]:
def rot_z(angle):
    return np.array([
            [np.cos(angle), -np.sin(angle), 0],
            [np.sin(angle),  np.cos(angle), 0],
            [            0,              0, 1]], dtype=np.float)

def calc_r_rect(R_left, R_right, C_left, C_right):
    # r_0_rect: first row of the rectification rotation matrix
    r_0_rect = C_right - C_left
    r_0_rect = r_0_rect / np.linalg.norm(r_0_rect) # R is othonormal -> r_0_rect must have unit length!
    r_0_rect = r_0_rect.T # row vector
    
    # k: auxiliary vector
    r_2_l = R_left[2:3, :]
    r_2_r = R_right[2:3, :]
    k = (r_2_l + r_2_r) / 2
    k = k.T

    # r_1_rect: second row of the rectification rotation matrix
    # r_1_rect is perpendicular to k and r_0_rect.
    r_1_rect = np.cross(k, r_0_rect.T, axis=0)
    r_1_rect = r_1_rect.T # row vector

    # r_2_rect: third row of the recitfication rotation matrix
    # r_2_rect is perpendicular to r_0_rect and r_1_rect.
    r_2_rect = np.cross(r_0_rect.T, r_1_rect.T, axis=0)
    r_2_rect = r_2_rect.T # row vector

    # build up the recitfication rotation matrix
    R_rect = np.vstack((r_0_rect, r_1_rect, r_2_rect))
    return R_rect

Please make also sure that you have saved the input images `C0_unrect.png` and `C1_unrect.png` in the same folder as the Jupyter Notebook.

In [ ]:
img_left  = cv2.imread("c0_unrect.png", cv2.IMREAD_GRAYSCALE)
img_right = cv2.imread("c1_unrect.png", cv2.IMREAD_GRAYSCALE)

## Calibration results

Two images have been recorded with with two cameras. The intrinsic and extrinsic parameters are given below and can be usually determined by a calibration procedure.

In [ ]:
# extrinsics
alpha = +5.0 / 180.0 * np.pi # z rotation left camera in radians
beta  = -3.0 / 180.0 * np.pi # z rotation right camera in radians
R_left = rot_z(alpha) # build up rotation matrix of left camera
R_right = rot_z(beta) # build up rotation matrix of right camera
C_left  = np.array([[  0, 0, 0]]).T # position of left camera (Assume CCS of left camera is congruent with WCS)
C_right = np.array([[0.3, 0, 0]]).T # position of right camera (baseline 30 cm = 0.3 m)
baseline = np.linalg.norm(C_right - C_left) # length of baseline: distance between the cameras

# instrinsics
focal_length = 1454.922678357857 # [px]
f_x = focal_length
f_y = focal_length
skew = 0
width = img_left.shape[1]
height = img_left.shape[0]
assert img_left.shape == img_right.shape # both images should have the same shape
c_x = (width - 1) / 2  # image center: x
c_y = (height - 1) / 2 # image center: y
K = np.array([
    [f_x, skew, c_x],
    [  0,  f_y, c_y],
    [  0,    0,   1]
], dtype=np.float)     # our intrinic camera matrix

## Rectification

The rectification transforms the images in such a way that the epipolar lines match. Please consider also the lecture sides of Chapter 7.

In [ ]:
R_left_inv = np.linalg.inv(R_left)    # R_left^-1
R_right_inv = np.linalg.inv(R_right)  # R_right^-1
K_inv = np.linalg.inv(K)              # K^-1

R_rect = calc_r_rect(R_left, R_right, C_left, C_right)
H_left_rect  = K.dot(R_rect.dot(R_left_inv.dot(K_inv)))   # rectification homography for left camera
H_right_rect = K.dot(R_rect.dot(R_right_inv.dot(K_inv)))  # rectification homography for right camera

# We can apply homographies on images (instead of point sets) with OpenCV's warpPerspective method...
img_left_rect  = cv2.warpPerspective(img_left,  H_left_rect,  (width, height))
img_right_rect = cv2.warpPerspective(img_right, H_right_rect, (width, height))

# annotate images (not important for exam)
img_left_vis  = np.copy(img_left)[::5, ::5] # downsample only for visualization (images are too big for screen)
img_right_vis = np.copy(img_right)[::5, ::5]
img_left_rect_vis  = np.copy(img_left_rect)[::5, ::5]
img_right_rect_vis = np.copy(img_right_rect)[::5, ::5]

# put text not important for exam
img_left_vis  = cv2.UMat.get(cv2.putText(img_left_vis, 'left',   (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0 , (255,), 1))
img_right_vis = cv2.UMat.get(cv2.putText(img_right_vis, 'right', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0 , (255,), 1))
img_left_rect_vis  = cv2.UMat.get(cv2.putText(img_left_rect_vis, 'left rect.',   (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0 , (255,), 1))
img_right_rect_vis = cv2.UMat.get(cv2.putText(img_right_rect_vis, 'right rect.', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0 , (255,), 1))


imgs_stacked = np.hstack((img_left_vis, img_right_vis))
imgs_rect_stacked = np.hstack((img_left_rect_vis, img_right_rect_vis))
vis = np.vstack((imgs_stacked, imgs_rect_stacked))

cv2.imshow("Rectification", vis)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Stereo correspondence

After rectification, we are able to search for pixel correspondences in the left and the right image. For that, we use SGBM [2]. The technique and the parameters of SGBM are not important for the examination in summer term '20 & winter term '20/'21. Feel free to play around with the parameters and look how the 3D result changes at the end...

In [ ]:
# SGBM default parameters from OpenCV's python example: stereo_match.py
window_size = 3                # default:   3 
min_disp = 170                 # default:  16
max_disp = 298                 # default: 112
num_disp = max_disp - min_disp # default: 112 - min_disp
assert num_disp % 16 == 0, "Attention: max_disp must be dividable by 16!"
block_size = 21                # default:  16
P1 = 8*3*window_size**2        # default:   8*3*window_size**2
P2 = 32*3*window_size**2       # default:  32*3*window_size**2
disp_left_right_max_diff = 1   # default:   1
pre_filter_cap = 0             # default:   0
uniq_ratio = 10                # default:  10
speckle_win_size = 300         # default: 100
speckle_range = 32             # default:  32

# helper object that performs the SGBM:
stereo = cv2.StereoSGBM_create(minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=block_size,
        P1=P1,
        P2=P2,
        disp12MaxDiff=disp_left_right_max_diff,
        preFilterCap=pre_filter_cap,
        uniquenessRatio=uniq_ratio,
        speckleWindowSize=speckle_win_size,
        speckleRange=speckle_range)

disp_map = stereo.compute(img_left_rect, img_right_rect).astype(np.float32) / 16.0 # the disparity map

# removing some invalid disparity values
invalid_map_a = (img_left_rect == 0)                        # black borders as an artefact of rectif. -> ignore
invalid_map_b = np.zeros_like(invalid_map_a, dtype=np.bool) 
invalid_map_b[:,:max_disp] = True                           # SGBM has always a invalid left boarder -> ignore
invalid_map = np.bitwise_or(invalid_map_a, invalid_map_b)   # merge invalid maps
disp_map[invalid_map] = -1                                  # we can set invalid disp. values to neg. values

## Triangulation

We can derive the depth map $\mathbf{Z}$ from the disparity map $\mathbf{D}$. The relation is given as:  
$Z = \frac{f\cdot b}{D}$, whereas $f$ and $b$ denote the focal length and the length of the baseline.

In [ ]:
depth_map = focal_length * baseline / disp_map
depth_map[invalid_map] = -1 # we can set invalid depth values to -1 meters

From the lecture we know that $\frac{x_\text{cam}}{z_\text{cam}} = \frac{x_\text{norm}}{1}$.  
Hence, $x_\text{cam} = x_\text{norm} \cdot \frac{z_\text{cam}}{1}$
and $y_\text{cam} = y_\text{norm} \cdot \frac{z_\text{cam}}{1}$.  
In order to derive the $\mathbf{X}_\text{norm}$, we can undo the affine transformation that usually transforms the $\mathbf{X}_\text{norm}$ to $\mathbf{X}_\text{img}$.

In [ ]:
# build up image points (not pixel values) of X_img
x_lin_space = np.linspace(0, width - 1, width)    # 0, 1, 2, ..., w-1
y_lin_space = np.linspace(0, height - 1, height)  # 0, 1, 2, ..., h-1

X_img_x, X_img_y = np.meshgrid(x_lin_space, y_lin_space)

# X_img_x:
# 0, 1, 2, ..., w-1
# 0, 1, 2, ..., w-1

# X_img_y:
#   0,   0,   0, ...,   0
#   1,   1,   1, ...,   1
#           ...
# h-1, h-1, h-1, ..., h-1

X_img_x = X_img_x.reshape((1, width * height)) # flatten the x meshgrid
X_img_y = X_img_y.reshape((1, width * height)) # flatten the y meshgrid
X_img_w = np.ones_like(X_img_x, dtype=np.float)    # corresponding homogeneous scaling factors for each point
X_img = np.vstack((X_img_x, X_img_y, X_img_w)) # stacking to a point matrix

# calculating X_norm
X_norm = K_inv.dot(X_img)
X_norm_inhom = X_norm[:2, :] / X_norm[2, :]

# calculating X_cam
z_cam = depth_map.reshape((1, width * height))
focal_length_of_X_norm = 1.0
X_cam_xy_inhom = X_norm_inhom * z_cam
X_cam_inhom = np.vstack((X_cam_xy_inhom, z_cam)) # containing x, y and z coordinates od all 3D points

## Visualization

This section is not important for the examination.

In [ ]:
# beautifying disparity map
disp_map_for_vis = disp_map.astype(np.float64)
disp_map_for_vis *= 0.5 # looks more pleasant...
disp_map_for_vis[disp_map_for_vis < 0 ] = 0 # draw invalid values with black pixels
disp_map_for_vis[disp_map_for_vis > (2**8 - 1) ] = 2**8 - 1 # There's no whiter than white...
disp_map_for_vis = disp_map_for_vis.astype(np.uint8)
disp_map_for_vis = disp_map_for_vis[::5, ::5]
disp_map_for_vis = cv2.UMat.get(cv2.putText(disp_map_for_vis, 'disparity',   (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0 , (255,), 1))

# beautifying depth map
depth_map_for_vis = np.copy(depth_map)
depth_map_for_vis *= (2**8 - 1) / np.max(depth_map)
depth_map_for_vis[depth_map_for_vis < 0 ] = 0 # draw invalid values with black pixels
depth_map_for_vis[depth_map_for_vis > (2**8 - 1) ] = 2**8 - 1 # There's no whiter than white...
depth_map_for_vis = depth_map_for_vis.astype(np.uint8)
depth_map_for_vis = depth_map_for_vis[::5, ::5]
depth_map_for_vis = cv2.UMat.get(cv2.putText(depth_map_for_vis, 'depth',   (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0 , (255,), 1))



# stacking results to one image
disp_and_depth = np.vstack((disp_map_for_vis, depth_map_for_vis))
overview = np.hstack((vis, disp_and_depth))

# show the results
cv2.imshow("overview", overview) # open window
cv2.waitKey(0) # wait until any key is pressed
cv2.destroyAllWindows() # close all windows

The plotted image consists of the following content:  
1st row: raw input images in grayscale  
2nd row: rectified images  
3rd row: disparity map and depth map  
  
Optionally: Write down a 3D point cloud. You can open the *Wavefront* file `room.obj` with Meshlab...

In [ ]:
### write the 3D point cloud

colors = img_left_rect.reshape((1, width * height)) # flatten the colors for simpler indexing
colors = colors.astype(np.float) / 255.0 # 0 -> 0 & 255 -> 1
invalid_points = invalid_map.reshape((1, width * height))

print("This might take some minutes...")
valid_indices = [i for i in range(width*height) if invalid_points[0, i] == False]
with open("room.obj", "w") as room:
    for i in valid_indices:
        x = X_cam_inhom[0, i]
        y = X_cam_inhom[1, i]
        z = X_cam_inhom[2, i]
        gray = colors[0, i]
        r = gray
        b = gray
        g = gray
        
        room.write("v " + str(x) + " " + str(y) + " " + str(z) + " " + str(r) + " " + str(g) + " " + str(b) + os.linesep)
        if i % 100 == 0 or i == (width * height - 1):
            status = (i+1) / (width * height) * 100
            print("status: %03.2d %%" % status, end='\r')
    print("status: 100 %")
print("Finished.")

### References

[1] B.-S. Hua, Q.-H. Pham, D. T. Nguyen, M.-K. Tran, L.-F. Yu, and S.-K. Yeung, ‘SceneNN: A Scene Meshes Dataset with aNNotations’, in 2016 Fourth International Conference on 3D Vision (3DV), Stanford, CA, USA, Oct. 2016, pp. 92–101, doi: 10.1109/3DV.2016.18.  
[2] H. Hirschmüller, ‘Accurate and Efficient Stereo Processing by Semi-Global Matching and Mutual Information’, in 2005 IEEE Computer Society Conference on Computer Vision and Pattern Recognition (CVPR’05), San Diego, CA, USA, 2005, vol. 2, pp. 807–814, doi: 10.1109/CVPR.2005.56.

